In [4]:
from function import get_db_philosopher

In [5]:
get_db_philosopher("Anne Conway")

{'label': 'Anne Conway',
 'wikiPageID': '2169543',
 'db_influence_record': {'influences': [],
  'influencedBy': [],
  'isInfluencesOf': [],
  'isInfluencedByOf': []},
 'abstract': '',
 'birthYear': '',
 'deathYear': '',
 'birthPlace': '',
 'deathPlace': '',
 'sameAs': ['wikidata:Anne Conway', 'https://global.dbpedia.org/id/4Q6Uj']}

get owl:sameAs, the information collected by DBpedia

In [ ]:
# we probably don't need this
def get_db_ent_resl(ent):
    # input individual ent allows us to update items individually

    db_url = "https://dbpedia.org/page/"+ent.replace(" ","_")
    page = requests.get(db_url)
    info_df = pd.read_html(page.content)[0].values

    

    for i in info_df:
        text = re.sub(' \t+', ';', i[1])
        if i[0] == "owl:sameAs":
            sameAs = text.split(";")
    
    return sameAs

In [ ]:
ent_info_list = pickle.load( open( "info_lists/db_philo_info_resolved_list.p", "rb" ) )
for item in ent_info_list:
    # print(get_db_ent_resl( item['entity']))
    item['dbpedia_info']['sameAs'] = get_db_ent_resl( item['entity'])

# check
COUNT = 0
for item in ent_info_list:
    try:
        if item['dbpedia_info']['sameAs']:
            COUNT = COUNT + 1
    except:
        continue


COUNT_3 = 0
for item in ent_info_list:
    try:
        if item['entity']:
            COUNT_3 = COUNT_3 + 1
    except:
        continue
print(COUNT_3)

if COUNT == COUNT_3:
    pickle.dump(ent_info_list, open( "db_philo_info_sameAs_list.p", "wb" ) )
else:
    print("Check")
    print(COUNT, COUNT_3)
    # print out the suspicious results
    for item in ent_info_list:
        try:
            if item['entity'] and not item['dbpedia_info']['sameAs']:
                # COUNT_3 = COUNT_3 + 1
                print(item)
        except:
            continue

# =====fix mistakes in a non-expensive way--update individuals
# for item in db_philo_info_list:
#     if item['entity'] == 'Dickinson Miller': # correct ent
#         item['entity'] = 'Dickinson S. Miller'
#         item['dbpedia_info']['sameAs'] = get_db_ent_resl( item['entity'])

#         item['wiki_url'] = 'https://en.wikipedia.org/wiki/'+item['entity'].replace(" ","_")
#         item['db_url'] = 'https://dbpedia.org/page/'+item['entity'].replace(" ","_")

#     elif item['entity'] == 'Gottfried Ploucquet':
#         item['entity'] = ''
#     elif item['entity'] == 'Ismael Quiles':
#         item['entity'] = ''
# for item in db_philo_info_list:
#     try:
#         if item['entity'] and not item['dbpedia_info']['sameAs']:
#             print(item)
#     except:
#         continue


KeyboardInterrupt: 

In [ ]:
ent_info_list
pickle.dump(ent_info_list, open( "info_lists/db_philo_info_sameAs_list.p", "wb" ) )

[{'entity': 'Lopamudra',
  'gender_listed': 0,
  'inclusion': 'main: excluded; women: included',
  'refered_name': 'Lopamudra',
  'wiki_url': 'https://en.wikipedia.org/wiki/Lopamudra',
  'db_url': 'https://dbpedia.org/page/Lopamudra',
  'dbpedia_info': {'label': 'Lopamudra',
   'wikiPageID': '7330387',
   'db_influence_record': {'influences': [],
    'influencedBy': [],
    'isInfluencesOf': [],
    'isInfluencedByOf': []},
   'abstract': 'Lopamudra (Sanskrit: लोपामुद्रा) also known as Kaushitaki and Varaprada was a philosopher according to ancient Vedic Indian literature. She was the wife of the sage Agastya who is believed to have lived in the Rigveda period (1950 BC-1100 BC) as many hymns have been attributed as her contribution to this Veda. She was not only the consort of Agastya but a Rishiki in her own right, as she was the well known Rishiki who visualized the "Hadi Panchadasi" mantra of the Srikul Shakta tradition of Hinduism. She was one of the prominent Brahmavadini. There a

Transform the birthdate to timestamp

In [ ]:
born_str = "12th century"
parse_born(born_str)

-61788528000

In [ ]:
# handle time
import re

born_patterns = list(map(lambda regex: re.compile(regex, re.IGNORECASE), [
    r"\d+\s*(BC|AD)",
    r"\d+\s*(january|february|march|april|may|june|july|august|september|october|november|december)\s*\d+",
    r"(january|february|march|april|may|june|july|august|september|october|november|december)\s*\d+,\s*\d+",
    r"\d+(st|nd|rd|th)\s*century\s*(AD|BC|BCE|CE)",
    r"\d+\s*(AD|BC|BCE|CE)?"
]))

number_pattern = re.compile(r"\d+", re.IGNORECASE)
month_pattern = re.compile(r"(january|february|march|april|may|june|july|august|september|october|november|december)", re.IGNORECASE)
month_nr = {'january': '01', 'february': '02', 'march': '03', 'april': '04', 'may': '05', 'june': '06', 'july': '07', 'august': '08', 'september': '09', 'october': '10', 'november': '11', 'december': '12'}

# handle time: clean and convert to string

def parse_born(born_str):
    born = None
    matched_pattern = None

    if born_str is not None:
        for pattern in born_patterns:
            match = pattern.search(born_str)
            if match is not None:
                matched_pattern = pattern
                born = match.group()
                break

    try:
        return date_str_to_int(born_str_to_date_str(born, matched_pattern))
    except:
        return None

    
def born_str_to_date_str(born_str, matched_pattern):
    if matched_pattern == born_patterns[0]:
        year_str = number_pattern.match(born_str).group()
        return year_sign(born_str) + year_str
    elif matched_pattern == born_patterns[1] or matched_pattern == born_patterns[2]:
        number_matches = re.findall(number_pattern, born_str)
        day_str = number_matches[0]
        if len(day_str) == 1:
            day_str = '0' + day_str
        year_str = number_matches[1]
        month_str = month_nr[re.findall(month_pattern, born_str)[0].lower()]
        return year_str + '-' + month_str + '-' + day_str
    elif matched_pattern == born_patterns[3]:
        century_str = number_pattern.match(born_str).group()
        return year_sign(born_str) + century_str + '00'
    elif matched_pattern == born_patterns[4]:
        year_str = number_pattern.match(born_str).group()
        return year_sign(born_str) + year_str
    else:
        return born_str

    
def year_sign(born_str):
    if 'BC' in born_str or 'BCE' in born_str:
        return '-'
    else:
        return ""

def date_str_to_int(date_str):
    datetime = np.datetime64(date_str)
    return int(datetime.astype('<M8[s]').astype(np.int64))

In [ ]:
def convert_timestamp(stime): # (int) timestamp = convert_timestampstime)
    # Unix epoch; stime is a string, return an int

    # why do we need to write a function?
    # time.mktime cannot handle time before 1970
    # time.mktime(datetime.datetime.strptime("1966-01-01", "%Y-%m-%d").timetuple())

    # if not stime:
    #     # if y,m,d = 0, 12pm => -2211710400
    #     return -2211710400

    # corner case: 0610-01-01; -586-01-01
    t = stime.split("-")[-3:]
    # to an array (y,m,d)
    y = int(t[0])
    m = int(t[1])
    d = int(t[2])

    epoch = datetime(1970, 1, 1)
    t = datetime(y, m, d)
    diff = t-epoch

    return diff.days * 24 * 3600 + diff.seconds

In [ ]:
for item in dbPhilo_info_list:
    # timestamp
    item["born"] = convert_timestamp(item["birthYear"])

deal with birthdate

Add missing birthdate? (It is not my job! IT is about research)

1. get from the list page - maybe more consistent
2. get from the abstract


- Circa (abbreviated c. or ca.) is a Latin-origin word meaning 'approximately'.
- fl. is used by historians as an abbreviation for "flourished." It is normally used only when dates of birth or death are unknown.
- AD stands for Anno Domini, Latin for “in the year of the Lord”, while BC stands for “before Christ”.
- B.C. (before Christ)
- The letters CE or BCE in conjunction with a year mean after or before year 1. CE is an abbreviation for Common Era. BCE is short for Before Common Era.


In [ ]:
db_philo_info_infludict_list = pickle.load( open("info_lists/db_philo_info_infludict_list.p", "rb") )
ent_info_list = copy.deepcopy(db_philo_info_infludict_list)
# philosophers who have no birthdate recorded
missing_born_ent_list = []
for item in ent_info_list:
    if not item['dbpedia_info']['birthYear']:
        # print(item['entity'], item['listed_info']['time'])
        missing_born_ent_list.append(item['entity'])
print(len(missing_born_ent_list))

1144


In [ ]:
# transform the time--let's only take into account year, NO DATE

for item in ent_info_list:
    item['listed_info']['time_transformed'] = ""
    listed_time = item['listed_info']['time']
    year = ""
    if item['entity'] == "":
        continue
    # if "–" in listed_time or "-" in listed_time or "—" in listed_time: # find digits
    #     continue
    # elif "born" in listed_time: # find digits
    #     continue

    # elif "century" in listed_time:
    #     continue

    elif "died" in listed_time:
        continue
    elif "fl." in listed_time: # ignore
        continue
    # elif "c." in listed_time or "circa" in listed_time or "ca." in listed_time:
    #     continue
    # else:
        # print(listed_time)
        
    
    four_digit = "1\d{3}" # should always start with 1
    three_digit = "\d{3}"
    try:
        year = year + re.findall(four_digit, listed_time)[0]
    except:
        try:
            year = year + re.findall(three_digit, listed_time)[0]
        except:
            year = ""
            # print(item['entity'],listed_time, "|","miss!")
    print(item['entity'],listed_time, "|",year)
    # item['listed_info']['time_transformed'] = year
    # if not item['dbpedia_info']['birthYear']:
    #     print(item['entity'], item['listed_info']['time'])
        
    

Lopamudra born 1100 BCE | 1100
Maitreyi born about 1000 BCE | 1000
Ghosha born vedic period | 
Gargi Vachaknavi born about 700 BCE | 700
Theano of Croton 6th century BCE | 
Aristoclea of Delphi 6th century BCE | 
Khujjuttarā 6th century BCE | 
Aspasia of Miletus approx. 470–400 BCE | 470
Arete of Cyrene 4th century BCE | 
Hipparchia of Maroneia c. 350 bc-c. 280 bc | 350
Catherine of Alexandria 282–305 | 282
Ptolemais of Cyrene 3rd century BCE | 
Aesara of Lucania 3rd century BCE | 
Diotima of Mantinea appears in Plato's Symposium | 
Ban Zhao c. 35–100 | 100
Sosipatra of Ephesus 4th century CE | 
Xie Daoyun before 340–after 399 | 340
Hypatia c. 360–415 CE | 360
Aedesia 5th century CE | 
Theodora 5th-6th century CE | 
Ubhaya Bharati 8th century | 
Héloïse d'Argenteuil 1090–1164 | 1090
Akka Mahadevi c.1130–1160 | 1130
Marguerite Porete 1250–1310 | 1250
Tullia d'Aragona c. 1510–1556 | 1510
Lalleshwari 1320–1392 | 1320
Catherine of Siena 1347–1380 | 1347
Moderata Fonte 1555–1592 | 1555
Hild